In [1]:
import pandas as pd
from datetime import date
import numpy as np

In [4]:
def limpiar_dataframe_inicial(df: pd.DataFrame) -> pd.DataFrame:
    """
    Realiza una limpieza general inicial en el DataFrame.

    Esta función toma un DataFrame, elimina los espacios en blanco
    de las columnas de tipo 'object' (texto), convierte la columna 'date'
    a formato datetime, la ordena cronológicamente, elimina filas con
    fechas nulas y extrae la fecha y la hora en nuevas columnas.

    Args:
        df (pd.DataFrame): El DataFrame de entrada que se va a limpiar.

    Returns:
        pd.DataFrame: El DataFrame procesado y limpio.
    """
    df_copia = df.copy()

    for col in df_copia.columns:
        if df_copia[col].dtype == 'object':
            df_copia[col] = df_copia[col].str.strip()

    df_copia['date'] = pd.to_datetime(df_copia['date'], dayfirst=True)
    df_copia = df_copia.sort_values(by='date', ascending=True)
    df_copia = df_copia.dropna(subset=['date'])

    df_copia['fecha'] = df_copia['date'].dt.date
    df_copia['hora'] = df_copia['date'].dt.time

    return df_copia

In [6]:
def limpiar_columnas_categoricas(df: pd.DataFrame) -> pd.DataFrame:
    """
    Limpia y estandariza las columnas categóricas del DataFrame.

    Esta función corrige valores mal escritos y maneja los datos nulos
    en las columnas 'WeekStatus', 'Day_of_week' y 'Load_Type'.
    Los valores incorrectos se reemplazan por su versión correcta y los
    nulos se rellenan utilizando el método 'forward fill'.

    Args:
        df (pd.DataFrame): DataFrame con las columnas categóricas a limpiar.

    Returns:
        pd.DataFrame: DataFrame con las columnas categóricas estandarizadas.
    """
    df_copia = df.copy()

    # Mapeo de correcciones para cada columna
    mapa_correcciones = {
        'WeekStatus': {
            'wEEKDAY': 'Weekday', 'wEEKEND': 'Weekend', 'NAN': np.nan
        },
        'Day_of_week': {
            'mONDAY': 'Monday', 'tUESDAY': 'Tuesday', 'wEDNESDAY': 'Wednesday',
            'tHURSDAY': 'Thursday', 'fRIDAY': 'Friday', 'sATURDAY': 'Saturday',
            'sUNDAY': 'Sunday', 'NAN': np.nan
        },
        'Load_Type': {
            'lIGHT_lOAD': 'Light_Load', 'mEDIUM_lOAD': 'Medium_Load',
            'mAXIMUM_lOAD': 'Maximum_Load', 'NAN': np.nan
        }
    }

    columnas_categoricas = ['WeekStatus', 'Day_of_week', 'Load_Type']

    for col in columnas_categoricas:
        df_copia[col] = df_copia[col].replace(mapa_correcciones[col])
        df_copia[col] = df_copia[col].ffill()
        df_copia[col] = df_copia[col].astype(str)

    return df_copia

In [7]:
def limpiar_columnas_numericas(df: pd.DataFrame) -> pd.DataFrame:
    """
    Limpia y estandariza un conjunto predefinido de columnas numéricas.

    La función identifica valores no numéricos en columnas que deberían serlo,
    los convierte a NaN (Not a Number) y luego utiliza el método 'forward fill'
    para imputar los valores faltantes. Finalmente, asegura que todas las
    columnas procesadas tengan el tipo de dato 'float'.

    Args:
        df (pd.DataFrame): DataFrame con columnas numéricas a procesar.

    Returns:
        pd.DataFrame: DataFrame con las columnas numéricas limpias y
                      en formato float.
    """
    df_copia = df.copy()

    columnas_numericas = [
        'Usage_kWh', 'Lagging_Current_Reactive.Power_kVarh',
        'Leading_Current_Reactive_Power_kVarh', 'CO2(tCO2)',
        'Lagging_Current_Power_Factor', 'Leading_Current_Power_Factor',
        'NSM', 'mixed_type_col'
    ]

    for col in columnas_numericas:
        # Forzar valores no numéricos a NaN
        df_copia[col] = pd.to_numeric(df_copia[col], errors='coerce')

        # Rellenar valores nulos
        df_copia[col] = df_copia[col].ffill()

        # Asegurar el tipo float
        df_copia[col] = df_copia[col].astype(float)

    return df_copia